In [49]:
import pandas as pd
import urllib
import re
# see https://www.utf8-chartable.de/unicode-utf8-table.pl?start=9728&number=128&names=-&utf8=dec
import unicodedata
import re



In [50]:
# Test regex on https://extendsclass.com/regex-tester.html#python

regexStartMultiVar = "[a-z][1-9]*\)"

stringMarkStartMultiVar = "#%"
regexStartMarkedMultiVar  = stringMarkStartMultiVar  + regexStartMultiVar

stringMarkEndMultiVar = "%#"
regexEndMarkedMultiVar = stringMarkEndMultiVar + regexStartMultiVar

def markMultiVar(string):
  returnString = ""
  for line in string.splitlines(False):
    tokenList = re.split(regexStartMultiVar, line, 1)
    if tokenList[0] == line:
      returnString += line + "\n"
    else:
      varString = line.replace(tokenList[1],'')
      # Does it make sense to mark the end of a line?
      #returnString += stringMarkStartMultiVar + varString + tokenList[1] + " " + stringMarkEndMultiVar + varString + "\n"
      returnString += stringMarkStartMultiVar + varString + tokenList[1] + "\n"

  return returnString




In [51]:
result =  markMultiVar("a2) 5... ♘a6 6. ♕c2 ♘xc5 7. a3 ♗xc3+ 8. ♕xc3 b6")
print (result)

#%a2) 5... ♘a6 6. ♕c2 ♘xc5 7. a3 ♗xc3+ 8. ♕xc3 b6



In [52]:
def normalizeC24String(c24String):

  def convertUnicode(string):
    if string[0:2] == "\\u":
      return chr(int(string[2:6], 16))
    else:
      return string

  def replaceUnicode(string):
    utfMapFileUrl = "https://raw.githubusercontent.com/littlecapa/chess24Parser/main/config/mapUTF2PGN.csv"
    mapList = pd.read_csv(utfMapFileUrl, sep=';')
    mapList = mapList.fillna("")
    for index, row in mapList.iterrows():
      string = string.replace(convertUnicode(row["UTF"]), row["pgnStr"])
    return string

  def findPgnToken(string):
    pgnTokenFileUrl = "https://raw.githubusercontent.com/littlecapa/chess24Parser/main/config/pgntoken.csv"
    pgnList = pd.read_csv(pgnTokenFileUrl, sep=';')
    pgnList = pgnList.iloc[pgnList.agg({"string":len}).sort_values('string', ascending=False).index]

    for index, row in pgnList.iterrows():
      string = string.replace(row["string"], row["pgnStr"])
    return string

  def cleanUp(string):
    regexMultipleNL = "\n(\n)+"
    return (re.sub(regexMultipleNL, "\n", string))
  
  c24String = replaceUnicode(c24String)
  c24String = findPgnToken(c24String)
  c24String = markMultiVar(c24String)
  c24String = cleanUp(c24String)

  return c24String


In [53]:
def Test_normalizeC24String():
  # Test Case 1
  c24String ="1. d4 d5+/= 2. c4+- c6-+ 3. ♘f3=/+ ♘f6 4. ♘c3 dxc4 5. a4! ♗f5? 6. e3!! e6?? 7. ♗xc4!? ♗b4?! 8. O-O= O-O 9. ♕e2"
  normString = normalizeC24String(c24String)
  
  if normString != "1. d4 d5$14 2. c4$18 c6$19 3. Nf3$15 Nf6 4. Nc3 dxc4 5. a4$28 Bf5$29 6. e3$3 e6$4 7. Bxc4$24 Bb4$25 8. O-O$10 O-O 9. Qe2\n":
    print("Error")
    print(normString)
  else:
    print("OK")

In [54]:
Test_normalizeC24String()

OK


In [55]:
# Tokenizer
regexWhiteMoveNr = "([0-9]+[\\.]{1})"
regexBlackMoveNr = "([0-9]+[\\.]{3})"
regexMoveNr = "(" + regexBlackMoveNr + "|" + regexWhiteMoveNr + ")"
regexNagOpt = "(\\$[0-9]+)?"
regexPiece  = "([K|Q|R|B|N])"
regexPromoted             = regexPiece + "?"
regexFile                 = "([a-h])"
regexRank                 = "([1-8])"
regexSquare               = regexFile + regexRank
regexXorFileRankSquareOpt = "(" + regexFile + "|" + regexRank + "|" + regexSquare + ")?"
regexCapture              = "[x]?"
regexCastleMove           = "(O\\-O|O\\-O\\-O)"
regexPieceMove            = regexPiece + regexXorFileRankSquareOpt + regexCapture + regexSquare
regexPawnPushMove         = regexSquare + regexPromoted
regexPawnCaptureMove      = regexFile + regexRank + "?" + regexCapture + regexPawnPushMove
regexChessOrMate          = "(\\+|#){0,1}"
regexMove                 = "(" + regexPieceMove + "|" + regexPawnPushMove + "|" + regexPawnCaptureMove + "|" + regexCastleMove + ")" + regexChessOrMate + regexNagOpt

# regexMoveinComment : wird beim Normalisieren verwendet,
# um Züge in Kommentaren zu finden

regexMoveinComment = "(" + regexPieceMove + ")"
moveInCommentTag = "#MICT#"



stringStartIfVar    = "%IF%"
regexStartIfVar     = "(" + stringStartIfVar + ")"


regexStartSingleVar = "( \\()"
regexEndSingleVar   = "(\\))"


regexMoveOrNrOrVar = "(" + regexMoveNr + "|" + regexMove + "|" + regexStartMarkedMultiVar + "|" + regexEndMarkedMultiVar + "|" + regexStartSingleVar + "|" + regexStartIfVar + "|" + regexEndSingleVar + "|(\\n)" + ")"





In [56]:
def testLeningrad():
  url = "https://raw.githubusercontent.com/littlecapa/chess24Parser/main/testdata/nimzoOddsEnds.c24"
  file = urllib.request.urlopen(url)
  c24String = ""

  for line in file:
    c24String += str(line.decode('utf-8')) + "\n"

  normString = normalizeC24String(c24String)
  print (normString)

testLeningrad()

1. d4  Nf6 2. c4 e6 3. Nc3 Bb4 4. Qb3
4. Bd2 O-O 5. e3
#%a) 5... b6 6. Nf3 Bb7 7. Bd3 d5 ( 7... d6 8. O-O Nbd7 9. a3 Bxc3 10. Bxc3 Ne4 ) 8. O-O
#%b) 5... d5 6. Nf3 b6 ( 6... Re8 7. Bd3 dxc4 8. Bxc4 ) 7. a3 Be7
4. Bg5 c5
#%a) 5. dxc5
#%a1) 5... h6 6. Bh4 ( 6. Bd2 ) 6... Qa5 7. Bxf6
#%a2) 5... Na6 6. Qc2 Nxc5 7. a3 Bxc3+ 8. Qxc3 b6
#%b) 5. d5 d6
#%b1) 6. e3 exd5 7. cxd5 Nbd7
#%b11) 8. Nf3 Qa5
#%b12) 8. Bd3 Qa5 9. Ne2 Nxd5 10. O-O Bxc3 11. bxc3 c4 12. Bc2 ( 12. Be4 N5f6 13. Bxf6 Nxf6 14. Bf3 Qc5 ; 12. Bxc4 N5b6 ) 12... O-O
#%b121) 13. Qd4 Nxc3
#%b122) 13. Ng3 Nxc3 14. Qh5 f5 15. e4 ( 15. Be7 Rf7 16. Nxf5 Ne5 17. Bxd6 Bxf5 18. Bxf5 Nf3+ 19. Qxf3 Qxf5 20. Qxf5 Ne2+ 21. Kh1 Rxf5 22. Rae1 Nc3 ) 15... Nxe4
#%b123) 13. Bh4 13... Nxc3 14. Nxc3 Qxc3 15. Rc1 Qa3 ( 15... Qe5 16. Bg3 Qe6 17. Qxd6 Qxd6 18. Bxd6 Rd8 19. Be4 Nf6 ) 16. Be7 Re8 17. Bxd6 Qa6 ( 17... Qxa2 ) 18. e4 Qc6
#%b13) 8. Bb5 8... h6 ( 8... a6 9. Bxd7+ Bxd7 10. Qf3 ) 9. Bh4 Bxc3+ 10. bxc3 O-O 11. Ne2 Nb6 12. O-O Nbxd5
#%b2) 6. f3 6..